In [8]:
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.preprocessing.image import load_img, array_to_img
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
titanic_orig = pd.read_csv('train.csv')
titanic_test_orig = pd.read_csv('test.csv')

In [5]:
def transform(data):    
    data['hasCabin'] = data['Cabin'].notna()
    data = data.drop(['PassengerId', 'Name', 'Ticket','Cabin'],axis=1)

    data = pd.get_dummies(data)
    data.Age = data.Age.fillna(data.Age.mean())
    data.Fare = data.Fare.fillna(data.Fare.mean())
    return data

In [6]:
titanic = transform(titanic_orig)
titanic_test = transform(titanic_test_orig)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(titanic.drop(['Survived'],axis=1),titanic['Survived'],test_size=0.2,random_state=0)

In [14]:
X_train

,Pclass,Age,SibSp,Parch,Fare,hasCabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
140,3,29.699118,0,2,15.2458,False,1,0,1,0,0
439,2,31.000000,0,0,10.5000,False,0,1,0,0,1
817,2,31.000000,1,1,37.0042,False,0,1,1,0,0
378,3,20.000000,0,0,4.0125,False,0,1,1,0,0
491,3,21.000000,0,0,7.2500,False,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
835,1,39.000000,1,1,83.1583,True,1,0,1,0,0
192,3,19.000000,1,0,7.8542,False,1,0,0,0,1
629,3,29.699118,0,0,7.7333,False,0,1,0,1,0
559,3,36.000000,1,0,17.4000,False,1,0,0,0,1


In [16]:
y_train=to_categorical(y_train,2)
y_test=to_categorical(y_test,2)

In [22]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(11,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(2,activation='softmax'))

In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               6144      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 1026      
Total params: 269,826
Trainable params: 269,826
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))/

Train on 712 samples, validate on 179 samples
Epoch 1/10
712/712 [==============================] - 0s 71us/step - loss: 0.3918 - accuracy: 0.8315 - val_loss: 0.4506 - val_accuracy: 0.7933
Epoch 2/10
712/712 [==============================] - 0s 63us/step - loss: 0.3840 - accuracy: 0.8315 - val_loss: 0.4252 - val_accuracy: 0.7877
Epoch 3/10
712/712 [==============================] - 0s 65us/step - loss: 0.3987 - accuracy: 0.8301 - val_loss: 0.4297 - val_accuracy: 0.7877
Epoch 4/10
712/712 [==============================] - 0s 66us/step - loss: 0.3799 - accuracy: 0.8343 - val_loss: 0.4750 - val_accuracy: 0.7821
Epoch 5/10
712/712 [==============================] - 0s 66us/step - loss: 0.4151 - accuracy: 0.8230 - val_loss: 0.5401 - val_accuracy: 0.7765
Epoch 6/10
712/712 [==============================] - 0s 63us/step - loss: 0.4083 - accuracy: 0.8343 - val_loss: 0.4428 - val_accuracy: 0.7933
Epoch 7/10
712/712 [==============================] - 0s 65us/step - loss: 0.3814 - accuracy: 0.

In [29]:
model.predict(titanic_test)

array([[9.06111479e-01, 9.38884914e-02],
       [7.89006114e-01, 2.10993901e-01],
       [9.99244213e-01, 7.55845977e-04],
       [8.67984354e-01, 1.32015646e-01],
       [5.33748984e-01, 4.66251045e-01],
       [7.84977973e-01, 2.15022057e-01],
       [1.84894428e-01, 8.15105617e-01],
       [8.83195341e-01, 1.16804689e-01],
       [2.59746701e-01, 7.40253270e-01],
       [9.30844963e-01, 6.91550970e-02],
       [8.64905238e-01, 1.35094732e-01],
       [7.94808984e-01, 2.05191031e-01],
       [1.41237481e-02, 9.85876262e-01],
       [8.90567124e-01, 1.09432854e-01],
       [7.26248510e-03, 9.92737472e-01],
       [8.48923624e-02, 9.15107667e-01],
       [8.08735728e-01, 1.91264242e-01],
       [7.27330565e-01, 2.72669464e-01],
       [5.51948309e-01, 4.48051691e-01],
       [8.95931244e-01, 1.04068696e-01],
       [7.84750760e-01, 2.15249285e-01],
       [5.58953345e-01, 4.41046655e-01],
       [2.43079867e-02, 9.75692034e-01],
       [7.94124007e-01, 2.05876023e-01],
       [3.444694

In [ ]:
0.